In [119]:
import boto3
import pandas as pd
s3=boto3.resource('s3')
import csv
import io
from io import StringIO

### Getting Keys to all the csv files

In [120]:
BUCKET_NAME ='imagineerawslohith'

#for key in s3.buckets(BUCKET_NAME).Key:

allFiles = s3.Bucket(BUCKET_NAME).objects.all()
file_keys=[]
for file in allFiles:
    file_keys.append(file.key)
print(file_keys)


['consumption data for aws/RefBldgFullServiceRestaurantNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgLargeOfficeNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgMidriseApartmentNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgPrimarySchoolNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgSmallHotelNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgStand-aloneRetailNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgStripMallNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv', 'consumption data for aws/RefBldgSuperMarketNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv']


### Creating a function to extract csv and convert them to dataframes

In [121]:

def read_s3_objects(bucket_name, key):
    response = s3.Object(bucket_name, key).get()
    data=response['Body'].read()
    df=pd.read_csv(io.BytesIO(data),encoding='utf8')
    return df

df0=read_s3_objects(BUCKET_NAME,file_keys[0])
df1=read_s3_objects(BUCKET_NAME,file_keys[1])
df2=read_s3_objects(BUCKET_NAME,file_keys[2])
df3=read_s3_objects(BUCKET_NAME,file_keys[3])
df4=read_s3_objects(BUCKET_NAME,file_keys[4])
df5=read_s3_objects(BUCKET_NAME,file_keys[5])
df6=read_s3_objects(BUCKET_NAME,file_keys[6])
df7=read_s3_objects(BUCKET_NAME,file_keys[7])


   

### Creating a list of Dataframes

In [122]:
Dfmain=[df0,df1,df2,df3,df4,df5,df6,df7]

### Merging all the Dataframe into a Single Dataframe DFM

In [123]:
DFM=pd.concat(Dfmain, axis=0 ,ignore_index=True)
File_name="Concatinated_DataFrame.csv"
DFM.to_csv(File_name)

In [124]:
DFM.head()

Date/Time  Electricity:Facility [kW](Hourly)  \
0   01/01  01:00:00                          22.547756   
1   01/01  02:00:00                          14.688283   
2   01/01  03:00:00                          14.714919   
3   01/01  04:00:00                          14.725921   
4   01/01  05:00:00                          14.881438   

   Fans:Electricity [kW](Hourly)  Cooling:Electricity [kW](Hourly)  \
0                       4.021388                          0.001463   
1                       0.000000                          0.000000   
2                       0.000000                          0.000000   
3                       0.000000                          0.000000   
4                       0.000000                          0.000000   

   Heating:Electricity [kW](Hourly)  InteriorLights:Electricity [kW](Hourly)  \
0                               0.0                                 4.589925   
1                               0.0                                 1.529975   
2                               0.0                                 1.529975   
3                               0.0                                 1.529975   
4                               0.0                                 1.529975   

   InteriorEquipment:Electricity [kW](Hourly)  Gas:Facility [kW](Hourly)  \
0                                      8.1892                  77.395159   
1                                      7.4902                   3.359880   
2                                      7.4902                   3.931953   
3                                      7.4902                   3.359880   
4                                      7.4902                   3.931959   

   Heating:Gas [kW](Hourly)  InteriorEquipment:Gas [kW](Hourly)  \
0                 67.705358                             3.33988   
1                  0.000000                             3.33988   
2                  0.000000                             3.33988   
3                  0.000000                             3.33988   
4                  0.000000                             3.33988   

   Water Heater:WaterSystems:Gas [kW](Hourly)  \
0                                    6.349921   
1                                    0.020000   
2                                    0.592073   
3                                    0.020000   
4                                    0.592079   

   Electricity:Facility [kW](Monthly)  Gas:Facility [kW](Monthly)  
0                                 NaN                         NaN  
1                                 NaN                         NaN  
2                                 NaN                         NaN  
3                                 NaN                         NaN  
4                                 NaN                         NaN

### Concatinating year to Date/Time column and changing the data type

In [125]:
DFM["Date/Time"]=DFM["Date/Time"].str.lstrip()
DFM["Date/Time"]=DFM["Date/Time"].apply(lambda x:"{}{}".format('2004/',x))


In [126]:
#DFM["Date/Time"]=pd.to_datetime(DFM["Date/Time"],infer_datetime_format=True)
DFM[['date','time']] = DFM['Date/Time'].str.split(expand=True)
DFM['Date/Time'] = (pd.to_datetime(DFM.pop('date'), format='%Y/%m/%d') + 
                  pd.to_timedelta(DFM.pop('time') + ''))


In [127]:
DFM["Date/Time"].head()

0   2004-01-01 01:00:00
1   2004-01-01 02:00:00
2   2004-01-01 03:00:00
3   2004-01-01 04:00:00
4   2004-01-01 05:00:00
Name: Date/Time, dtype: datetime64[ns]

### Checking Data Types of the columns

In [128]:
DFM.dtypes

Date/Time                                     datetime64[ns]
Electricity:Facility [kW](Hourly)                    float64
Fans:Electricity [kW](Hourly)                        float64
Cooling:Electricity [kW](Hourly)                     float64
Heating:Electricity [kW](Hourly)                     float64
InteriorLights:Electricity [kW](Hourly)              float64
InteriorEquipment:Electricity [kW](Hourly)           float64
Gas:Facility [kW](Hourly)                            float64
Heating:Gas [kW](Hourly)                             float64
InteriorEquipment:Gas [kW](Hourly)                   float64
Water Heater:WaterSystems:Gas [kW](Hourly)           float64
Electricity:Facility [kW](Monthly)                   float64
Gas:Facility [kW](Monthly)                           float64
dtype: object

### Uploading the Dataframe to S3 Bucket

In [129]:
s3.meta.client.upload_file(File_name,BUCKET_NAME,'All_Consumption_Data.csv')